### XGBoost and Optuna

In [1]:
import pandas as pd
import numpy as np

# the GBM used
import xgboost as xgb

from sklearn.model_selection import KFold

# to encode categoricals
from sklearn.preprocessing import LabelEncoder

# see utils.py
from utils import add_features, rmsle, train_encoders, apply_encoders

import optuna

In [2]:
# globals and load train dataset

FILE_TRAIN = "train.csv"
FILE_TEST = "test.csv"

In [3]:
# load train dataset
data_orig = pd.read_csv(FILE_TRAIN)

#
# add features
#
data_extended = add_features(data_orig)

In [4]:
# ok, we will treat as categorical: holiday, hour, season, weather, workingday, year

In [5]:
all_columns = data_extended.columns

# cols to be ignored
# atemp and temp are strongly correlated (0.98) we're taking only one
del_columns = ["datetime", "casual", "registered", "temp"]

TARGET = "count"
cat_cols = ["season", "holiday", "workingday", "weather", "windspeed", "hour", "year"]
num_cols = list(set(all_columns) - set([TARGET]) - set(del_columns) - set(cat_cols))
features = sorted(cat_cols + num_cols)

print("All columns:", len(all_columns))
print("Ignored columns:", len(del_columns))
print("Target:", len([TARGET]))
print("Categorical columns:", len(cat_cols))
print("Numerical columns:", len(num_cols))
print("All the features", len(features))

All columns: 14
Ignored columns: 4
Target: 1
Categorical columns: 7
Numerical columns: 2
All the features 9


In [6]:
# drop ignored columns
data_used = data_extended.drop(del_columns, axis=1)

In [7]:
# let's code categorical
le_list = train_encoders(data_used)

# coding
data_used = apply_encoders(data_used, le_list)

# define indexes for cat_cols
# cat boost want indexes
cat_columns_idxs = [i for i, col in enumerate(features) if col in cat_cols]

train for coding: season 
train for coding: weather 
train for coding: year 

Coding: season 
Coding: weather 
Coding: year 


In [8]:
# general
FOLDS = 7
SEED = 4321
# train for longer, see if I can reduce RMSLE from 0.7
N_TRIALS = 60
STUDY_NAME = "gbm2"


X = data_used[features].values
y = data_used[TARGET].values

In [9]:
# for XGBoost

dtrain = xgb.DMatrix(X, label=y)

In [10]:
#
# Here we define what we do using Optuna
#
def objective(trial):

    # tuning on max_depth, n_estimators for the example
    dict_params = {
        "verbosity": 0,
        "objective": "reg:squarederror",
        "learning_rate": trial.suggest_loguniform("learning_rate", low=1e-4, high=1e-2),
        "eval_metric": "rmse",
        "max_depth": trial.suggest_int("max_depth", 4, 10),
    }

    # needed only for XGBoost
    n_rounds = trial.suggest_categorical(
        "num_boost_round", [1000, 2000, 3000, 4000, 5000]
    )
    dict_params["num_boost_round"] = n_rounds

    history = xgb.cv(
        params=dict_params,
        dtrain=dtrain,
        nfold=FOLDS,
        seed=SEED,
        # as far as I see we need this for XGBoost
        num_boost_round=n_rounds,
    )

    # take the last
    rmse = round(history["test-rmse-mean"].values[-1], 4)

    return rmse

In [11]:
# launch Optuna Study

study = optuna.create_study(study_name=STUDY_NAME, direction="minimize")

study.optimize(objective, n_trials=N_TRIALS)

[I 2022-03-08 17:09:17,851] A new study created in memory with name: gbm2
[I 2022-03-08 17:11:13,476] Trial 0 finished with value: 50.3394 and parameters: {'learning_rate': 0.001338134232361574, 'max_depth': 10, 'num_boost_round': 2000}. Best is trial 0 with value: 50.3394.
[I 2022-03-08 17:13:29,968] Trial 1 finished with value: 43.3153 and parameters: {'learning_rate': 0.0035008245434434105, 'max_depth': 6, 'num_boost_round': 5000}. Best is trial 1 with value: 43.3153.
[I 2022-03-08 17:16:44,695] Trial 2 finished with value: 74.0092 and parameters: {'learning_rate': 0.00041053116341917425, 'max_depth': 9, 'num_boost_round': 4000}. Best is trial 1 with value: 43.3153.
[I 2022-03-08 17:18:27,477] Trial 3 finished with value: 81.1703 and parameters: {'learning_rate': 0.0005458897213907421, 'max_depth': 7, 'num_boost_round': 3000}. Best is trial 1 with value: 43.3153.
[I 2022-03-08 17:19:01,876] Trial 4 finished with value: 67.5695 and parameters: {'learning_rate': 0.002089996778149659, 

[I 2022-03-08 18:27:59,176] Trial 38 finished with value: 42.2876 and parameters: {'learning_rate': 0.006416749991473487, 'max_depth': 7, 'num_boost_round': 3000}. Best is trial 32 with value: 42.1909.
[I 2022-03-08 18:28:27,577] Trial 39 finished with value: 64.2987 and parameters: {'learning_rate': 0.0026085758322579294, 'max_depth': 6, 'num_boost_round': 1000}. Best is trial 32 with value: 42.1909.
[I 2022-03-08 18:30:16,111] Trial 40 finished with value: 42.4865 and parameters: {'learning_rate': 0.008189411921340458, 'max_depth': 6, 'num_boost_round': 4000}. Best is trial 32 with value: 42.1909.
[I 2022-03-08 18:31:53,146] Trial 41 finished with value: 42.1639 and parameters: {'learning_rate': 0.006809030784424057, 'max_depth': 7, 'num_boost_round': 3000}. Best is trial 41 with value: 42.1639.
[I 2022-03-08 18:33:29,966] Trial 42 finished with value: 42.3007 and parameters: {'learning_rate': 0.007380600278838579, 'max_depth': 7, 'num_boost_round': 3000}. Best is trial 41 with value

In [12]:
study.best_params

{'learning_rate': 0.006809030784424057,
 'max_depth': 7,
 'num_boost_round': 3000}

In [13]:
# visualize trials as an ordered Pandas df
df = study.trials_dataframe()

result_df = df[df["state"] == "COMPLETE"].sort_values(by=["value"], ascending=True)

# best on top
result_df.head()

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_max_depth,params_num_boost_round,state
41,41,42.1639,2022-03-08 18:30:16.112852,2022-03-08 18:31:53.146314,0 days 00:01:37.033462,0.006809,7,3000,COMPLETE
32,32,42.1909,2022-03-08 18:17:24.281973,2022-03-08 18:19:00.406822,0 days 00:01:36.124849,0.009800,7,3000,COMPLETE
54,54,42.2155,2022-03-08 18:50:29.827178,2022-03-08 18:52:07.305875,0 days 00:01:37.478697,0.007884,7,3000,COMPLETE
25,25,42.2163,2022-03-08 18:04:06.046882,2022-03-08 18:06:14.912744,0 days 00:02:08.865862,0.005898,7,4000,COMPLETE
26,26,42.2249,2022-03-08 18:06:14.914417,2022-03-08 18:08:24.832114,0 days 00:02:09.917697,0.005331,7,4000,COMPLETE


### estimate RMSLE
* it is the metric used in Kaggle competition

In [14]:
# make an estimation of rmsle on entire dataset

kf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

avg_rmsle = 0.0

#
# at each iteration you get a different set of indexes
# from which you get different samples for train and validation dataset
#
for i, (train_idx, valid_idx) in enumerate(kf.split(data_used)):
    print("Processing fold:", i + 1)

    # here we split the DataFrame, using the indexes for the fold
    data_train = data_used.iloc[train_idx]
    data_valid = data_used.iloc[valid_idx]

    x_train = data_train[features].values
    y_train = data_train[TARGET].values
    x_valid = data_valid[features].values
    y_valid = data_valid[TARGET].values

    # it is important to pass explicitely n_estimators (only xgboost)
    model = xgb.XGBRegressor(
        n_estimators=study.best_params["num_boost_round"], **study.best_params
    )

    model.fit(x_train, y_train)

    y_valid_preds = model.predict(x_valid)

    # clip to zero (to avoid nan for rmsle)
    y_valid_preds = np.where(y_valid_preds >= 0, y_valid_preds, 0)

    avg_rmsle += rmsle(y_valid_preds, y_valid) / float(FOLDS)

print()
print("Avg. RMSLE:", round(avg_rmsle, 4))
print()

Processing fold: 1
Processing fold: 2
Processing fold: 3
Processing fold: 4
Processing fold: 5
Processing fold: 6
Processing fold: 7

Avg. RMSLE: 0.405



### Train the model on entire train set and save

In [15]:
%%time
model = xgb.XGBRegressor(
    n_estimators=study.best_params["num_boost_round"], **study.best_params
)

model.fit(x_train, y_train)

CPU times: user 52.2 s, sys: 87.9 ms, total: 52.3 s
Wall time: 14.6 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.006809030784424057,
             max_delta_step=0, max_depth=7, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=3000, n_jobs=4,
             num_boost_round=3000, num_parallel_tree=1, predictor='auto',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [16]:
model.save_model("xgboost.txt")